In [1]:
import os
import numpy as np
from pandas import read_csv
from tqdm import tqdm

## Section 1: Task performance metrics

### 1.1 Compute permutation statistics

In [2]:
from scipy.stats import ttest_ind
np.random.seed(47404)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Define variables of interest.
cols = ['accuracy','points','ws','ls','perseveration','beta','eta_p','eta_n','kappa']

## Define number of bootstraps.
n_iter = 5000

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Permutation statistics.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Load and prepare data.
metrics = read_csv(os.path.join('data', 'metrics.csv'))
correlates = read_csv(os.path.join('data', 'correlates.csv'))

## Merge DataFrames.
data = correlates.merge(metrics[['subject','platform','infreq']])
data = data.set_index('infreq')

## Binarize infrequency failures.
data.index = np.where(data.index > 0, 1, 0)

## Restrict to relevant columns.
data = data[cols]

## Compute means by group.
gb = data.groupby(level=0).mean().T
mu = (gb[0] - gb[1]).values

## Compute observed statistics.
t_obs = np.zeros(len(data.columns))
for i, col in enumerate(data.columns):
    t_obs[i], _ = ttest_ind(data.loc[0, col], data.loc[1, col])
    
## Compute null statistics.
t_null = np.zeros((n_iter, t_obs.size))
for n in tqdm(range(n_iter)):
    
    ## Permute labels.
    data.index = np.random.permutation(data.index)
    
    ## Compute null statistics.
    for i, col in enumerate(data.columns):
        t_null[n,i], _ = ttest_ind(data.loc[0, col], data.loc[1, col])
        
## Compute p-values.
p_val = np.mean(np.abs(t_obs) < np.abs(t_null), axis=0)

## Compute effect sizes.
d_val = t_obs * np.sqrt(np.sum(1 / data.index.value_counts()))

## Compute confidence interval. 
lb = mu - 1.96 * (mu / t_obs)
ub = mu + 1.96 * (mu / t_obs)

## Add to table.
gb['tval'] = t_obs; gb['pval'] = p_val; gb['dval'] = d_val
gb['lb'] = lb; gb['ub'] = ub

100%|██████████| 5000/5000 [00:45<00:00, 111.06it/s]


### 1.2 Supplementary tables

In [3]:
## Update columns.
gb = gb.rename(columns={0: 'Attentive', 1:'C/IE'})
gb['95% CI'] = gb.apply(lambda x: '[%0.3f, %0.3f]' %(x.lb, x.ub), 1)
gb = gb.drop(columns=['lb','ub'])

## Update rows.
gb = gb.rename(index={'accuracy': 'Acc', 'points': 'Pts', 'ws': 'WS', 'ls': 'LS',
                      'perseveration': 'Pers', 'beta': r'$\beta$', 'eta_p': r'$\eta_p$', 
                      'eta_n': r'$\eta_n$', 'kappa': r'$\kappa$'})

## Convert to LaTeX.
print(gb.round(3).to_latex())

\begin{tabular}{lrrrrrl}
\toprule
{} &  Attentive &    C/IE &   tval &   pval &   dval &            95\% CI \\
\midrule
Acc      &      0.587 &   0.532 &  4.008 &  0.000 &  0.492 &    [0.028, 0.082] \\
Pts      &     50.163 &  47.729 &  2.376 &  0.019 &  0.292 &    [0.426, 4.441] \\
WS       &      0.898 &   0.776 &  5.387 &  0.000 &  0.662 &    [0.077, 0.165] \\
LS       &      0.609 &   0.751 & -5.335 &  0.000 & -0.655 &  [-0.195, -0.090] \\
Pers     &      0.245 &   0.259 & -1.505 &  0.139 & -0.185 &   [-0.033, 0.004] \\
\$\textbackslash beta\$  &      6.754 &   4.082 &  5.404 &  0.000 &  0.664 &    [1.702, 3.640] \\
\$\textbackslash eta\_p\$ &      0.643 &   0.551 &  2.846 &  0.007 &  0.350 &    [0.029, 0.157] \\
\$\textbackslash eta\_n\$ &      0.738 &   0.784 & -1.516 &  0.120 & -0.186 &   [-0.106, 0.013] \\
\$\textbackslash kappa\$ &     -0.069 &  -0.218 &  3.729 &  0.000 &  0.458 &    [0.071, 0.227] \\
\bottomrule
\end{tabular}



<ipython-input-3-878cd200d3d3>:12: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(gb.round(3).to_latex())
